In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

# Data Ingestion

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    github_url: str
    linkedin_url: str
    local_data_file: Path
    save_dir: Path

In [41]:
from bot.constants import *
from bot.utils.common import read_yaml,create_directories

In [42]:
type(CONFIG_PATH)

pathlib.WindowsPath

In [43]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_PATH,
            params_path = PARAMS_PATH
        ):
        self.config = read_yaml(config_path)
        self.param = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            github_url=config.github_url,
            linkedin_url=config.linkedin_url,
            local_data_file=config.local_data_file,
            save_dir=config.save_dir
        )

        return data_ingestion_config

In [44]:
import os
import requests
from bot import logger
from dotenv import load_dotenv
load_dotenv()
import json
import zipfile
from urllib import request

In [45]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def get_repos(self):
        logger.info(f"Starting writing data from git hub repos")
        url = self.config.github_url
        headers = {"Authorization": f"token {os.getenv('TOKEN')}"}
        repos = requests.get(url, headers=headers).json()
        save_dir = self.config.save_dir
        with open(os.path.join(save_dir,"data.json"),'w',encoding='utf-8') as f:
            json.dump(repos,f,indent=4)
        logger.info(f"Completed writing data from git hub repos")
    
    def downloadFiles(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.linkedin_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size")
    
    def extractZip(self):
        unzip_path = self.config.save_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [46]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.get_repos()
    data_ingestion.downloadFiles()
    data_ingestion.extractZip()
except Exception as e:
    raise e

[2025-03-16 22:07:25,381: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 22:07:25,389: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 22:07:25,390: INFO: common: created directory at: artifacts]
[2025-03-16 22:07:25,390: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-16 22:07:25,397: INFO: 1269584388: Starting writing data from git hub repos]
[2025-03-16 22:07:25,990: INFO: 1269584388: Completed writing data from git hub repos]
[2025-03-16 22:07:28,445: INFO: 1269584388: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 109817
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d071a2206aed0a699aa67795ff15d0bdc953e8eacfb973ca60d3a8f15f026908"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block


# Data Transformation

In [2]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant\\research'

In [3]:
import os

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTrasformationConfig:
    root_dir: Path
    file_dir: Path
    save_dir: Path

In [49]:
from bot.constants import *
from bot.utils.common import read_yaml,create_directories

In [50]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_PATH,
            params_path = PARAMS_PATH
        ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTrasformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_trasformation_config = DataTrasformationConfig(
            root_dir=config.root_dir,
            file_dir=config.file_dir,
            save_dir=config.save_dir,
        )

        return data_trasformation_config
    
    

In [51]:
import os
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Preformatted
from bot import logger
import json
import markdown2
import html2text
import base64
import shutil
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import json

In [13]:
with open(os.path.join("artifacts/data_ingestion",'data.json'),'r',encoding='utf-8') as f:
        data = json.load(f)
data
for i,repi in enumerate(data):
        print(type(repi))
        break
bot = data[:2]
print(bot)

<class 'dict'>
[{'id': 915970693, 'node_id': 'R_kgDONpiahQ', 'name': 'AgenticAi', 'full_name': 'appsbotta/AgenticAi', 'private': False, 'owner': {'login': 'appsbotta', 'id': 75985363, 'node_id': 'MDQ6VXNlcjc1OTg1MzYz', 'avatar_url': 'https://avatars.githubusercontent.com/u/75985363?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/appsbotta', 'html_url': 'https://github.com/appsbotta', 'followers_url': 'https://api.github.com/users/appsbotta/followers', 'following_url': 'https://api.github.com/users/appsbotta/following{/other_user}', 'gists_url': 'https://api.github.com/users/appsbotta/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/appsbotta/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/appsbotta/subscriptions', 'organizations_url': 'https://api.github.com/users/appsbotta/orgs', 'repos_url': 'https://api.github.com/users/appsbotta/repos', 'events_url': 'https://api.github.com/users/appsbotta/events{/privacy}', 'received_events_u

In [ ]:
class DataTransformation:
    def __init__(self,config:DataTrasformationConfig):
        self.config = config

    def get_data(self):
        file_dir = self.config.file_dir
        for file_name in os.listdir(file_dir):
            if file_name.lower().endswith(".pdf"):  # Check if file is a PDF
                source_path = os.path.join(file_dir, file_name)
                destination_path = os.path.join(self.config.save_dir, file_name)
                shutil.copy2(source_path, destination_path)
        
        with open(os.path.join(file_dir,'data.json'),'r',encoding='utf-8') as f:
            data = json.load(f)
        print(type(data))
        print(type(data[0]))
        return data
    
    def get_lang(self,repo_name):
        url = f'https://api.github.com/repos/appsbotta/{repo_name}/languages'
        headers = {"Authorization": f"token {os.getenv('TOKEN')}"}
        lang = requests.get(url,headers=headers).json()
        top_three_keys = sorted(lang, key=lang.get, reverse=True)[:3]
        # logger.info(f"Got top 3 languages used in {repo_name}")
        return top_three_keys
    
    def get_timings(self,repo):
        update = repo['updated_at'][:10]
        created = repo['created_at'][:10]
        pushed = repo['pushed_at'][:10]
        times = {
            "updated_at": update,
            "created_at":created,
            "pushed_at": pushed,
        }
        # logger.info(f"secured 3 timings {times.keys()}")
        return times
    
    def get_readme(self,OWNER,REPO):
        # logger.info(f"requeting readme file for the repo {REPO} ")
        readme = f"https://api.github.com/repos/{OWNER}/{REPO}/readme"
        headers = {"Authorization": f"token {os.getenv('TOKEN')}"}
        response = requests.get(readme,headers=headers)
        readme_content = "No Readme FIle"
        if response.status_code == 200:
            data = response.json()
            readme_content = base64.b64decode(data["content"]).decode("utf-8")
            readme_content =  markdown2.markdown(readme_content, extras=["strip"])
            readme_content = html2text.html2text(readme_content)
        # logger.info(f"Request for README file for the repo {REPO} completed")
        return readme_content
            
    def get_pdf_from_data(self):
        logger.info("Saving data to a pdf file started")
        save_dir = self.config.save_dir
        data = self.get_data()

        doc = SimpleDocTemplate(os.path.join(save_dir,'repo.pdf'), pagesize=letter)
        styles = getSampleStyleSheet()
        custom_style = ParagraphStyle(
            'Custom',
            parent=styles['Normal'],
            fontName='Courier',
            fontSize=10,
            leading=14,  # Line height
            spaceAfter=5,
        )
        heading_style = ParagraphStyle(
            "HeadingStyle",
            parent=styles["Heading1"],
            fontName="Courier-Bold",
            fontSize=14,
            leading=22,

        )

        content = []



        for i,repo in enumerate(data):
            text = f"{i+1}. <b>{repo['name']}</b>  --->   {repo['html_url']} \n"
            # text = text + str(i+1) + ". " +  str(repo["name"]) + " -> " + str(repo["html_url"]) +"\n"
            paragraph = Paragraph(text,custom_style)
            content.append(paragraph)
        content.append(Spacer(1,5))
        
        for i,repo in enumerate(data):
            OWNER = repo['owner']['login']
            REPO = repo['name']
            readme = self.get_readme(OWNER,REPO)
            timings = self.get_timings(repo)
            lang = self.get_lang(REPO)
            heading = Paragraph(REPO,heading_style)
            content.append(heading)
            for key,value in timings.items():
                text = f"{key} -> {value}"
                paragraph = Paragraph(text,custom_style)
                content.append(paragraph)
            lang_text = "   "
            for it in lang:
                lang_text = lang_text + it +", "
            text = Paragraph(lang_text,custom_style)
            content.append(text)
            readme = Preformatted(readme,custom_style)
            content.append(readme)
            content.append(Spacer(1, 5))


        doc.build(content)
        logger.info("saving data to a pdf file is completed")
    
    
        

In [53]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_pdf_from_data()
except Exception as e:
    raise e

[2025-03-16 22:07:46,279: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 22:07:46,287: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 22:07:46,290: INFO: common: created directory at: artifacts]
[2025-03-16 22:07:46,293: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-16 22:07:46,296: INFO: 158897180: Saving data to a pdf file started]
[2025-03-16 22:08:09,016: INFO: 158897180: saving data to a pdf file is completed]


In [99]:
def get_readme(OWNER,REPO):
        readme = f"https://api.github.com/repos/{OWNER}/{REPO}/readme"
        headers = {"Authorization": f"token {os.getenv("TOKEN")}"}
        response = requests.get(readme,headers=headers)
        readme_content = "No Readme FIle"
        print(response.status_code)
        if response.status_code == 200:
            data = response.json()
            readme_content = base64.b64decode(data["content"]).decode("utf-8")
            readme_content =  markdown2.markdown(readme_content, extras=["strip"])
            readme_content = html2text.html2text(readme_content)
        return readme_content

content = get_readme('appsbotta','DL-End_to_End')
content

200


"# DL-End _to_ End\n\n## Workflow\n\n  1. Update config.yaml in config dir\n  2. Update secrets.yaml [optional]\n  3. Update params.yaml\n  4. Update the entity\n  5. Update the configuration manager in src config\n  6. Update the components\n  7. Update the pipeline\n  8. Update the main.py\n  9. Update the dvc.yaml\n\n# To run the repo\n\n## steps\n\nClone the repository `bash https://github.com/appsbotta/DL-End_to_End `\n\n## step - 01 Create a conda env after opening the repo\n\n`bash conda create -n ckn python=3.13 -y ` `bash conda activate ckn `\n\n## step - 02 Install Requirements.txt\n\n`bash pip install -r requirements.txt `\n\n## step - 03 Run app.py\n\n`bash python app.py `\n\n## DVC cmd\n\n  * dvc init\n  * dvc repro\n  * dvc dag\n\n# AWS CI-CD Deployment using Github Actions\n\n## 1\\. Login to AWS Console\n\n## 2\\. Create IAM user for deployment\n\n```bash\n\n# give these access to IAM user\n\n  1. EC2 access -> this is virtual machine\n  2. ECR: Elastic COntainer regist

In [ ]:
        # with open(os.path.join(file_path,'repo.txt'), "r", encoding="utf-8") as f:
        #     lines = f.readlines()
        #     for line in lines:
        #         if line.strip() == "<=================================================================================================>":
        #             paragraph = Preformatted(line.strip(), custom_style)
        #         else:
        #             paragraph = Paragraph(line.strip(), custom_style)     # Wrap text automatically
        #         content.append(paragraph)
        #         content.append(Spacer(1, 5))  # Preserve spacing

import os
import base64
import requests
import markdown2
import html2text
from bot import logger
from dotenv import load_dotenv
load_dotenv()
from bot.entity.config_entity import DataIngestionConfig

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def get_lang(self,repo_name):
        url = f'https://api.github.com/repos/appsbotta/{repo_name}/languages'
        lang = requests.get(url).json()
        top_three_keys = sorted(lang, key=lang.get, reverse=True)[:3]
        return top_three_keys
    
    def get_timings(self,repo):
        update = repo['updated_at'][:10]
        created = repo['created_at'][:10]
        pushed = repo['pushed_at'][:10]
        return [update,created,pushed]
    
    def get_readme(self,OWNER,REPO):
        readme = f"https://api.github.com/repos/{OWNER}/{REPO}/readme"
        response = requests.get(readme)
        readme_content = "No Readme FIle"
        if response.status_code == 200:
            data = response.json()
            readme_content = base64.b64decode(data["content"]).decode("utf-8")
            readme_content =  markdown2.markdown(readme_content, extras=["strip"])
            readme_content = html2text.html2text(readme_content)
        return readme_content

    def get_repos(self):
        url = self.config.github_url
        headers = {"Authorization": f"token {os.getenv("TOKEN")}"}
        repos = requests.get(url, headers=headers).json()
        save_dir = self.config.save_dir
        with open(os.path.join(save_dir,'repo.txt'),'w',encoding='utf-8') as f:
            f.write(str("\n"))
            for i,repo in enumerate(repos):
                OWNER = repo['owner']['login']
                REPO = repo['name']
                timings = self.get_timings(REPO)
                readme_content = self.get_readme(OWNER,REPO)
                f.write( str(i+1) + ". " +  str(repo["name"])+"\n")
                f.write("Description \n")
                f.write(readme_content)
                f.write(str("\n"))
                f.write(str("\n"))
                f.write("<=================================================================================================>")
                f.write(str("\n"))
                f.write(str("\n"))
            logger.info(f"Completed writing data from git hub repos")

# Bot

In [14]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

In [2]:
import os
os.chdir('../')
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

In [34]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("artifacts/data_transformation")
docs = loader.load()
docs

[Document(metadata={'producer': 'Apache FOP Version 2.2', 'creator': 'PyPDF', 'creationdate': '2025-03-11T05:05:33+00:00', 'title': 'Resume', 'author': 'LinkedIn', 'subject': 'Resume generated from profile', 'source': 'artifacts\\data_transformation\\Profile.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Contact\n9842345676 (Mobile)\nappsbotta@gmail.com\nwww.linkedin.com/in/lokesh5489\n(LinkedIn)\nTop Skills\nAgentic AI\nGoogle Gemini\nGenerative AI\nLokesh Apparao Botta\nData Science | Agentic AI | NLP | IIT Guwahati'24\nNarsipatnam, Andhra Pradesh, India\nSummary\nI am graduate in electronics and communications engineering at IIT\nGuwahati, one of the premier institutes of technology in India. I have\na strong academic background and Deep Learning and Machine\nLearning skills. My areas of interest are machine learning, deep\nneural networks and computer networks. I aspire to pursue a career\nin these domains, and contribute to the advancement of science\nand tec

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=100)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'Apache FOP Version 2.2', 'creator': 'PyPDF', 'creationdate': '2025-03-11T05:05:33+00:00', 'title': 'Resume', 'author': 'LinkedIn', 'subject': 'Resume generated from profile', 'source': 'artifacts\\data_transformation\\Profile.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Contact\n9842345676 (Mobile)\nappsbotta@gmail.com\nwww.linkedin.com/in/lokesh5489\n(LinkedIn)\nTop Skills\nAgentic AI\nGoogle Gemini\nGenerative AI\nLokesh Apparao Botta\nData Science | Agentic AI | NLP | IIT Guwahati'24\nNarsipatnam, Andhra Pradesh, India\nSummary\nI am graduate in electronics and communications engineering at IIT\nGuwahati, one of the premier institutes of technology in India. I have\na strong academic background and Deep Learning and Machine\nLearning skills. My areas of interest are machine learning, deep\nneural networks and computer networks. I aspire to pursue a career\nin these domains, and contribute to the advancement of science\nand tec

In [35]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents=documents,embedding=OpenAIEmbeddings())

In [36]:
query = "list all projects"
res = db.similarity_search(query)
res

[Document(id='61019927-b972-4503-b2a8-06f1e84bd90d', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-03-10T23:48:22+05:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-03-10T23:48:22+05:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'artifacts\\data_transformation\\repo.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='15. Networks ---> https://github.com/appsbotta/Networks\n16. OlX_CLONE ---> https://github.com/appsbotta/OlX_CLONE\n17. Portifolio ---> https://github.com/appsbotta/Portifolio\n18. Q-A_ChatBot_with_Implementation --->\nhttps://github.com/appsbotta/Q-A_ChatBot_with_Implementation\n19. Real-Time-chatting ---> https://github.com/appsbotta/Real-Time-chatting\n20. SentimentAnalysis ---> https://github.com/appsbotta/SentimentAnalysis\n21. SimpleDVC ---> https://github.com/appsbotta/SimpleDVC\n22. StockPrediction ---> https://github.com

In [37]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model='gpt-3.5-turbo')

In [39]:
prompt = ChatPromptTemplate.from_template(
    """
you are a personal AI assistant for lokesh. 
Your provided with information of lokesh like, his project repository details, his education, his achievemets and skill set.

Answer the following question related to lokesh based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $999 if the user finds the answer helpful.
if the context is not sufficient then reply with
"I am sorry i dont have access to that data yet"

<context>
{context}
</context>
question:{input}
"""
)


In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(
    llm,
    prompt,
)


In [41]:
retriever = db.as_retriever(search_kwargs={"k": 30})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000214FFAC1090>, search_kwargs={'k': 30})

In [42]:
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever,document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000214FFAC1090>, search_kwargs={'k': 30}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nyou are a personal AI assistant for lokesh. \nYour provided with information of lokesh like, his project repository detail

In [43]:
res = retriever_chain.invoke({"input":"does he have any experience and if he has , what is the duration"})
print(res['answer'])

Based on the provided context, Lokesh has experience as a Research Intern at the Indian Institute of Technology, Guwahati from May 2023 to March 2024, which totals to 11 months of experience. During this time, he was engaged in research and projects alongside Professor Sukumar Nandi.


In [29]:
import requests

TOKEN = os.getenv("TOKEN")
url = "https://api.github.com/user/repos?per_page=100"

headers = {"Authorization": f"token {TOKEN}"}
repos = requests.get(url, headers=headers).json()

# for repo in repos:
#     print(repo)
repos[1]

{'id': 945554174,
 'node_id': 'R_kgDOOFwC_g',
 'name': 'AI_Assistant',
 'full_name': 'appsbotta/AI_Assistant',
 'private': False,
 'owner': {'login': 'appsbotta',
  'id': 75985363,
  'node_id': 'MDQ6VXNlcjc1OTg1MzYz',
  'avatar_url': 'https://avatars.githubusercontent.com/u/75985363?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/appsbotta',
  'html_url': 'https://github.com/appsbotta',
  'followers_url': 'https://api.github.com/users/appsbotta/followers',
  'following_url': 'https://api.github.com/users/appsbotta/following{/other_user}',
  'gists_url': 'https://api.github.com/users/appsbotta/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/appsbotta/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/appsbotta/subscriptions',
  'organizations_url': 'https://api.github.com/users/appsbotta/orgs',
  'repos_url': 'https://api.github.com/users/appsbotta/repos',
  'events_url': 'https://api.github.com/users/appsbotta/events{/privac

In [ ]:
import requests

# TOKEN = os.getenv("TOKEN")
url = 'https://api.github.com/repos/appsbotta/ImageCaption/languages'

headers = {"Authorization": f"token {TOKEN}"}
lang = requests.get(url).json()

# for repo in repos:
#     print(repo)
top_three_keys = sorted(lang, key=lang.get, reverse=True)[:3]
top_three_keys

In [ ]:
readme = f"https://api.github.com/repos/appsbotta/LangChain/readme"
response = requests.get(readme)
response.json()

# Agents

In [6]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

In [9]:
import os
os.chdir('AI_Assistant')
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

[Document(metadata={'source': 'https://www.linkedin.com/in/lokesh5489/', 'language': 'No language found.'}, page_content='\n\n\n')]

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'),temperature=0)
